# Job Shop Scheduling at westaflex

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))
from pyomo.environ import *
from pyomo.gdp import *
from pandas_ods_reader import read_ods
import pandas as pd

## 1 Modelling the Optimization Problem

### 1.1 Machines
There are seven machines called "Wickelrohrmaschine" which work in parallel and can produce different tube sizes.

In [ ]:
MACHINES = {
    'm1': {'name': '1531'},
    'm2': {'name': '1532'},
    'm3': {'name': '1533'},
    'm4': {'name': '1534'},
    'm5': {'name': '1535'},
    'm6': {'name': '1536'},
    'm7': {'name': '1537'},
}

In [ ]:
### 1.2 Jobs
The Jobs are the orders to be produced. A Job is characterized by:
- a unique key
- a due time by which all tasks should be finished

In [ ]:
def parse_machine_number(df, column):
    """
    Function extracts the machine number of a expression in the format
    "Wickelrohrmaschine 1234" and gives back only the machines with a
    number expression.

    Parameters
    ----------
    df : pd.DataFrame
        dataframe with a column which contains a string expression
         for the machine
    column: string
        Name of the column that contains information about the machine

    Returns
    ---------
    pd.DataFrame
        Updated dataframe with new columns 'machine' and 'machine_id
    """
    #df = df[df.Nummer Wickel-rohrmaschine.str.startswith('Wickelrohrmaschine')]
    df['machine_id'] = df.apply(lambda row: str(row[column][-4:]).strip(),
                             axis=1)
    return df

In [ ]:
path = "../data/Arbeitsplanung_steuerung_Wickelmaschinen_alt.ods"
sheet_idx = 14
df = read_ods(path, sheet_idx)
df = df.rename(columns=df.iloc[10])
df = df.rename(columns={df.columns[0]:'Nichts'})
df = df.drop('Nichts', axis=1)
df = df.drop(np.arange(12))
df = df.reset_index(drop=True)
df = df[['Fertigungsauf-tragsnummer', 'Teilenummer', 
         'Auftragsmenge',  'Nummer Wickel-rohrmaschine', 'Werkzeug-nummer', 'Rüstzeit für WKZ/Materialwechsel', 'Rüstzeit für Coilwechsel', 'Maschinen-laufzeit', 'Fertigungszeit pro Mengeneinheit', 'Spätester Starttermin']]
df = parse_machine_number(df, 'Nummer Wickel-rohrmaschine')

In [ ]:
df_jobs = df[['Fertigungsauf-tragsnummer', 'Spätester Starttermin]]
jobs = df_jobs.to_dict('index')

### 1.3 Task
A task is the production of a job on a machine. 
It is characterized by:
- job
- machine
- duration 
- tool

In [ ]:
df_tasks = df[['Fertigungsauf-tragsnummer', 'machine_id', 'Maschinen-laufzeit', 'Werkzeug-nummer']]
tasks = df_tasks.to_dict('index')